In [10]:
import tensorflow as tf
import src.infovae as infovae

In [11]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
nn = infovae.InfoVAE(1, 16, 3)

x = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)
x_ = nn(x)

recon_loss, latent_loss = nn.make_losses(x, x_)
loss = recon_loss+latent_loss

In [13]:
train_summaries = [
    tf.summary.scalar('train/loss/recon', recon_loss),
    tf.summary.scalar('train/loss/latent', latent_loss),
    tf.summary.histogram('latents', nn.z),
    tf.summary.image('train/recon', x_),
]
test_summaries = [
    tf.summary.scalar('test/loss/recon', recon_loss),
    tf.summary.scalar('test/loss/latent', latent_loss),
    tf.summary.image('test/recon', x_),
]

train_merged = tf.summary.merge(train_summaries)
test_merged = tf.summary.merge(test_summaries)

In [14]:
opt = tf.train.AdamOptimizer()
train_step = opt.minimize(loss)
saver = tf.train.Saver()

In [15]:
logdir = '/tmp/infovae/2'
!rm -rf {logdir}

In [16]:
epochs = 10

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logdir, sess.graph)
    
    for i in range(epochs*1000):
        batch_x, _ = mnist.train.next_batch(50)
        _, train_summ = sess.run([train_step, train_merged], 
                                 feed_dict={x: infovae.InfoVAE.preprocess(batch_x)})
        
        if i % 10 == 0:
            writer.add_summary(train_summ, i)
        
        if i % 100 == 0:
            L, test_summ = sess.run([loss, test_merged], 
                                                feed_dict={x: 
                        infovae.InfoVAE.preprocess(mnist.test.images[:100, ...])})
            print('\rStep: {} Loss: {}'.format(i, L), end='', flush=True)
            writer.add_summary(test_summ, i)
    save_path = saver.save(sess, logdir+"/infovae.ckpt")

Step: 2300 Loss: 0.099832683801651

In [ ]:
save_path